In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model as skl
from regression import Regressors as reg

In [2]:
# Creating a dataset from the CSV file
data = pd.read_csv("weather_data.csv")
data

,Date,Maximum Temperature (°C),Minimum Temperature (°C),Mean Temperature (°C),Sunshine Duration (min),Shortwave Radiation (MJ/m²),Precipitation (mm),Maximum Relative Humidity (%),Minimum Relative Humidity (%),Mean Relative Humidity (%),Maximum Sea Level Pressure (hPa),Minimum Sea Level Pressure (hPa),Mean Sea Level Pressure (hPa),Maximum Wind Speed (m/s),Minimum Wind Speed (m/s),Mean Wind Speed (m/s),Wind Direction Dominant (°)
0,02/01/2008,2.610529,-4.039472,-0.716971,220.386890,5.264172,0.000000,96,59,73.500000,1022.9,1012.3,1018.27094,5.536244,0.400000,3.426518,110.169304
1,03/01/2008,5.630528,-1.179471,1.312195,149.133350,4.194036,0.000000,97,66,79.291664,1012.2,1004.9,1007.71246,6.673080,0.900000,4.628796,111.911970
2,04/01/2008,9.200529,-1.619471,3.012196,132.000000,4.972608,0.000000,96,60,80.000000,1014.2,1007.6,1010.79987,5.292448,1.500000,3.371137,114.188460
3,05/01/2008,10.950529,2.600528,7.090530,0.000000,2.124252,1.500000,89,57,70.583336,1015.1,1008.6,1012.06670,6.946222,1.421267,3.831986,201.784740
4,06/01/2008,8.540529,5.860529,7.061361,134.042560,2.159496,8.800001,98,76,90.250000,1019.7,1006.1,1013.62915,7.102817,0.900000,3.537873,246.239300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5385,30/09/2022,16.190529,6.340528,11.649693,307.843630,15.103656,0.000000,93,53,73.875000,1014.0,1007.2,1010.52910,1.964688,0.316228,1.089594,283.240500
5386,01/10/2022,16.310530,8.130529,12.479695,41.554966,5.920992,0.500000,86,61,73.750000,1019.9,1013.5,1015.44165,9.411164,1.552418,4.244933,237.491330
5387,02/10/2022,19.560530,14.260529,17.118862,480.000000,13.254948,0.000000,88,62,75.708336,1022.8,1020.2,1021.20416,8.547514,1.565247,5.617112,263.698400
5388,03/10/2022,18.200530,11.390529,14.677612,440.240400,13.985458,0.000000,93,59,78.083336,1026.3,1023.3,1024.75430,6.537584,0.282843,2.786462,347.471200


In [3]:
# Let's represent data as an array of floats
# As the data is ordered by date, the Date column is not important, so we get rid of it
data_array = np.array(data.drop(columns="Date").values)
data_array

array([[ 2.61052850e+00, -4.03947160e+00, -7.16971460e-01, ...,
         4.00000000e-01,  3.42651750e+00,  1.10169304e+02],
       [ 5.63052850e+00, -1.17947140e+00,  1.31219540e+00, ...,
         9.00000000e-01,  4.62879600e+00,  1.11911970e+02],
       [ 9.20052900e+00, -1.61947130e+00,  3.01219560e+00, ...,
         1.50000000e+00,  3.37113700e+00,  1.14188460e+02],
       ...,
       [ 1.95605300e+01,  1.42605290e+01,  1.71188620e+01, ...,
         1.56524750e+00,  5.61711170e+00,  2.63698400e+02],
       [ 1.82005300e+01,  1.13905290e+01,  1.46776120e+01, ...,
         2.82842730e-01,  2.78646250e+00,  3.47471200e+02],
       [ 2.09105280e+01,  8.40052900e+00,  1.37413630e+01, ...,
         1.16619040e+00,  2.96270280e+00,  1.18800026e+02]])

In [5]:
# We'll also need each column as a separate array

max_temp = np.array(data["Maximum Temperature (°C)"])
min_temp = np.array(data["Minimum Temperature (°C)"])
mean_temp = np.array(data["Mean Temperature (°C)"])

sunshine = np.array(data["Sunshine Duration (min)"])
radiation = np.array(data["Shortwave Radiation (MJ/m²)"])
precipitation = np.array(data["Precipitation (mm)"])

max_humidity = np.array(data["Maximum Relative Humidity (%)"])
min_humidity = np.array(data["Minimum Relative Humidity (%)"])
mean_humidity = np.array(data["Mean Relative Humidity (%)"])

max_pressure = np.array(data["Maximum Sea Level Pressure (hPa)"])
min_pressure = np.array(data["Minimum Relative Humidity (%)"])
mean_pressure = np.array(data["Mean Sea Level Pressure (hPa)"])

max_wind_speed = np.array(data["Maximum Wind Speed (m/s)"])
min_wind_speed = np.array(data["Minimum Wind Speed (m/s)"])
mean_wind_speed = np.array(data["Mean Wind Speed (m/s)"])
wind_direction = np.array(data["Wind Direction Dominant (°)"])